<a href="https://colab.research.google.com/github/seanvw/TR_Trade_1/blob/main/Share_dealing_v6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To deepseek-r1-distill-llama-70b with Groq

Write a python application to make calculations of profit as APR for each Share held in a Portfolio.

Include a demonstation of it's usage.

You can use the current share price for a given ticker symbol using yfinance.

A share can be bought and sold within a day or over a longer period, months or years.

It is very important that a share can be bought and sold within the same day, therefore the calaculation of days should not be an integer, but rather a floating point number.

When a share is bought, the date and time is captured to the minute.

When a share is sold, the date and time is captured to the minute.

The application should be able to calculate APR based on a combination of sold shares (realised profit/loss) together with the current value of shares currently held within a portfolio

The application should report the realised profit/loss of shares no longer owned, together with its bought and sold datetimes and the days elapsed between purchase and sale. Additionally the effective APR of the share sale should be reported.

The application should report the unrealised profit/loss of shares still held, together with its bought date and time. Additionally the effective APR of the share sale should be reported.

Within the application, all date and times should be human readable

The demonstation of usage should give examples of realised share dealings held for 0.5 day, 1 day, 5 days and 6 months respectively

The demonstation of usage should give examples of unrealised share purchases

Respond with the code only! Nothing else!

In [1]:
import yfinance as yf
from datetime import datetime

class Share:
    def __init__(self, ticker, quantity, buy_price, buy_datetime, sell_price=None, sell_datetime=None):
        self.ticker = ticker
        self.quantity = quantity
        self.buy_price = buy_price
        self.buy_datetime = buy_datetime
        self.sell_price = sell_price
        self.sell_datetime = sell_datetime

    def calculate_time_elapsed(self):
        if self.sell_datetime:
            time_diff = self.sell_datetime - self.buy_datetime
            days = time_diff.days + time_diff.seconds / (24 * 3600)
            return days
        else:
            current_time = datetime.now()
            time_diff = current_time - self.buy_datetime
            days = time_diff.days + time_diff.seconds / (24 * 3600)
            return days

    def calculate_apr(self):
        if self.sell_datetime:
            days = self.calculate_time_elapsed()
            profit = (self.sell_price - self.buy_price) * self.quantity
            apr = (profit / (self.buy_price * self.quantity)) / (days / 365) * 100
            return apr
        else:
            current_price = yf.Ticker(self.ticker).info['currentPrice']
            days = self.calculate_time_elapsed()
            unrealized_profit = (current_price - self.buy_price) * self.quantity
            apr = (unrealized_profit / (self.buy_price * self.quantity)) / (days / 365) * 100
            return apr

class Portfolio:
    def __init__(self):
        self.shares = []

    def add_share(self, share):
        self.shares.append(share)

    def generate_report(self):
        realized_report = []
        unrealized_report = []

        for share in self.shares:
            if share.sell_datetime:
                days = share.calculate_time_elapsed()
                profit = (share.sell_price - share.buy_price) * share.quantity
                apr = share.calculate_apr()
                realized_report.append({
                    'ticker': share.ticker,
                    'buy_datetime': share.buy_datetime.strftime('%Y-%m-%d %H:%M'),
                    'sell_datetime': share.sell_datetime.strftime('%Y-%m-%d %H:%M'),
                    'days': round(days, 2),
                    'profit': round(profit, 2),
                    'apr': round(apr, 2)
                })
            else:
                current_price = yf.Ticker(share.ticker).info['currentPrice']
                unrealized_profit = (current_price - share.buy_price) * share.quantity
                apr = share.calculate_apr()
                unrealized_report.append({
                    'ticker': share.ticker,
                    'buy_datetime': share.buy_datetime.strftime('%Y-%m-%d %H:%M'),
                    'current_price': round(current_price, 2),
                    'unrealized_profit': round(unrealized_profit, 2),
                    'apr': round(apr, 2)
                })

        return realized_report, unrealized_report

# Demonstration of usage
portfolio = Portfolio()

# Realized shares
portfolio.add_share(Share(
    'AAPL',
    10,
    150.0,
    datetime(2023, 10, 1, 9, 30),
    155.0,
    datetime(2023, 10, 1, 16, 0)
))

portfolio.add_share(Share(
    'MSFT',
    5,
    300.0,
    datetime(2023, 10, 2, 10, 0),
    305.0,
    datetime(2023, 10, 7, 10, 0)
))

portfolio.add_share(Share(
    'GOOG',
    2,
    2500.0,
    datetime(2023, 4, 1, 10, 0),
    2600.0,
    datetime(2023, 10, 1, 10, 0)
))

# Unrealized shares
portfolio.add_share(Share(
    'AMZN',
    3,
    1800.0,
    datetime(2023, 10, 1, 10, 0)
))

# Generate reports
realized, unrealized = portfolio.generate_report()

# Print reports
print("Realized Transactions:")
for transaction in realized:
    print(f"""
    Ticker: {transaction['ticker']}
    Buy Time: {transaction['buy_datetime']}
    Sell Time: {transaction['sell_datetime']}
    Days Held: {transaction['days']}
    Profit: ${transaction['profit']}
    APR: {transaction['apr']}%
    """)

print("\nUnrealized Holdings:")
for holding in unrealized:
    print(f"""
    Ticker: {holding['ticker']}
    Buy Time: {holding['buy_datetime']}
    Current Price: ${holding['current_price']}
    Unrealized Profit: ${holding['unrealized_profit']}
    APR: {holding['apr']}%
    """)

Realized Transactions:

    Ticker: AAPL
    Buy Time: 2023-10-01 09:30
    Sell Time: 2023-10-01 16:00
    Days Held: 0.27
    Profit: $50.0
    APR: 4492.31%
    

    Ticker: MSFT
    Buy Time: 2023-10-02 10:00
    Sell Time: 2023-10-07 10:00
    Days Held: 5.0
    Profit: $25.0
    APR: 121.67%
    

    Ticker: GOOG
    Buy Time: 2023-04-01 10:00
    Sell Time: 2023-10-01 10:00
    Days Held: 183.0
    Profit: $200.0
    APR: 7.98%
    

Unrealized Holdings:

    Ticker: AMZN
    Buy Time: 2023-10-01 10:00
    Current Price: $196.24
    Unrealized Profit: $-4811.3
    APR: -62.15%
    
